In [1]:
import pandas as pd
import sqlite3

In [2]:
def get_best_performing_circuits(conn):
    sql_query = '''
    WITH DriverRacePoints AS (
        SELECT
            r.driverId,
            ra.year,
            r.raceId,
            SUM(points) AS total_points
        FROM
            results r
        JOIN
            races ra ON r.raceId = ra.raceId
        WHERE
            ra.year <> 2023
        GROUP BY
            r.driverId,
            ra.year,
            r.raceId
    ),
    MaxPointsPerDriverYear AS (
        SELECT
            driverId,
            year,
            MAX(total_points) AS max_points
        FROM
            DriverRacePoints
        GROUP BY
            driverId,
            year
    )
    SELECT
        drp.driverId,
        drp.year,
        ci.name as best_performing_circuit_name
    FROM
        DriverRacePoints drp
    JOIN
        MaxPointsPerDriverYear mppdy ON drp.driverId = mppdy.driverId
        AND drp.year = mppdy.year
    JOIN
        races ra ON drp.raceId = ra.raceId
    JOIN
        circuits ci ON ra.circuitId = ci.circuitId
        AND drp.total_points = mppdy.max_points
    GROUP BY
        drp.driverId,
        drp.year;
    '''

    df = pd.read_sql_query(sql_query, conn)

    return df
